In [8]:
import numpy as np
import pandas as pd
import re
import nltk
from nltk.tokenize import sent_tokenize, word_tokenize, PunktSentenceTokenizer, punkt, TweetTokenizer
from nltk.corpus import stopwords,state_union
from nltk.stem.snowball import SnowballStemmer
from nltk.stem import WordNetLemmatizer
import sys
import string
import json
from collections import Counter
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.base import TransformerMixin, BaseEstimator
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import BernoulliNB, MultinomialNB
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt
import preprocessor as p

In [4]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
from xgboost import XGBClassifier
import xgboost as xgb
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn import svm
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer

In [5]:
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
data = pd.read_csv("./training.1600000.processed.noemoticon.csv",encoding='latin-1')
DATASET_COLUMNS = ["Sentiment", "ids", "date", "flag", "user", "Text"]
data.columns = DATASET_COLUMNS
positif_data = data[data.Sentiment==4].iloc[:5000,:]
print(positif_data.shape)
neutral_data = data[data.Sentiment==2].iloc[:5000,:]
print(neutral_data.shape)
negative_data = data[data.Sentiment==0].iloc[:5000,:]
print(negative_data.shape)
data = pd.concat([positif_data,neutral_data,negative_data],axis = 0)
print(data.shape)

(5000, 6)
(0, 6)
(5000, 6)
(10000, 6)


In [114]:
data.head()

,Sentiment,ids,date,flag,user,Text
799999,4,1467822272,Mon Apr 06 22:22:45 PDT 2009,NO_QUERY,ersle,I LOVE @Health4UandPets u guys r the best!!
800000,4,1467822273,Mon Apr 06 22:22:45 PDT 2009,NO_QUERY,becca210,im meeting up with one of my besties tonight! ...
800001,4,1467822283,Mon Apr 06 22:22:46 PDT 2009,NO_QUERY,Wingman29,"@DaRealSunisaKim Thanks for the Twitter add, S..."
800002,4,1467822287,Mon Apr 06 22:22:46 PDT 2009,NO_QUERY,katarinka,Being sick can be really cheap when it hurts t...
800003,4,1467822293,Mon Apr 06 22:22:46 PDT 2009,NO_QUERY,_EmilyYoung,@LovesBrooklyn2 he has that effect on everyone


In [6]:
data.shape

(10000, 6)

In [9]:
# Removing Twitter Handles (@user)
data['SentimentText'] = data['Text'].str.replace("@", "")
# Removing links
data['SentimentText'] = data['SentimentText'].str.replace(r"http\S+", "")
# Removing Punctuations, Numbers, and Special Characters
data['SentimentText'] = data['SentimentText'].str.replace("[^a-zA-Z]", " ")
# Remove stop words
stopwords=nltk.corpus.stopwords.words('english')
def remove_stopwords(text):
    clean_text=' '.join([word for word in text.split() if word not in stopwords])
    return clean_text
data['SentimentText'] = data['SentimentText'].apply(lambda text : remove_stopwords(text.lower()))
# Text Tokenization and Normalization
data['SentimentText'] = data['SentimentText'].apply(lambda x: nltk.word_tokenize(x))
# Now let’s stitch these tokens back together
data['SentimentText'] = data['SentimentText'].apply(lambda x: ' '.join([w for w in x]))
# Removing small words
data['SentimentText'] = data['SentimentText'].apply(lambda x: ' '.join([w for w in x.split() if len(w)>3]))
count_vectorizer = CountVectorizer(stop_words='english')
cv = count_vectorizer.fit_transform(data['SentimentText'])

data

,Sentiment,ids,date,flag,user,Text,SentimentText
799999,4,1467822272,Mon Apr 06 22:22:45 PDT 2009,NO_QUERY,ersle,I LOVE @Health4UandPets u guys r the best!!,love health uandpets guys best
800000,4,1467822273,Mon Apr 06 22:22:45 PDT 2009,NO_QUERY,becca210,im meeting up with one of my besties tonight! ...,meeting besties tonight cant wait girl talk
800001,4,1467822283,Mon Apr 06 22:22:46 PDT 2009,NO_QUERY,Wingman29,"@DaRealSunisaKim Thanks for the Twitter add, S...",darealsunisakim thanks twitter sunisa meet sho...
800002,4,1467822287,Mon Apr 06 22:22:46 PDT 2009,NO_QUERY,katarinka,Being sick can be really cheap when it hurts t...,sick really cheap hurts much real food plus fr...
800003,4,1467822293,Mon Apr 06 22:22:46 PDT 2009,NO_QUERY,_EmilyYoung,@LovesBrooklyn2 he has that effect on everyone,lovesbrooklyn effect everyone
...,...,...,...,...,...,...,...
4995,0,1468963112,Tue Apr 07 04:36:43 PDT 2009,NO_QUERY,itikecil,a friend broke his promises..,friend broke promises
4996,0,1468963184,Tue Apr 07 04:36:43 PDT 2009,NO_QUERY,JoanneDuran,@gjarnling I am fine thanks - tired,gjarnling fine thanks tired
4997,0,1468963325,Tue Apr 07 04:36:46 PDT 2009,NO_QUERY,ewinsk,trying to keep my eyes open..damn baking,trying keep eyes open damn baking
4998,0,1468963547,Tue Apr 07 04:36:50 PDT 2009,NO_QUERY,Abbeyluvselvie,why the hell is it snowing,hell snowing


In [11]:
def most_used_words(text):
    tokens = word_tokenize(text)
    frequency_dist = nltk.FreqDist(tokens)
    print("There is %d different words" % len(set(tokens)))
    return sorted(frequency_dist, key=frequency_dist.__getitem__, reverse=True)

def stop_words():
    mw = most_used_words(train_data.text.str.cat())
    most_words = []
    for w in mw:
        if len(most_words) == 1000:
            break
        if w in stopwords.words("english"):
            continue
        else:
            most_words.append(w)
    print(sorted(most_words))

def stem_tokenize(text):
    stemmer = SnowballStemmer("english")
    stemmer = WordNetLemmatizer()
    return [stemmer.lemmatize(token) for token in word_tokenize(text)]

def lemmatize_tokenize(text):
    lemmatizer = WordNetLemmatizer()
    return [lemmatizer.lemmatize(token) for token in word_tokenize(text)]
class TextPreProc(BaseEstimator, TransformerMixin):
    def __init__(self, use_mention=False):
        self.use_mention = use_mention

    def fit(self, X, y=None):
        return self

    def transform(self, X, y=None):
        # We can choose between keeping the mentions
        # or deleting them
        if self.use_mention:
            X = X.str.replace(r"@[a-zA-Z0-9_]* ", " @tags ")
        else:
            X = X.str.replace(r"@[a-zA-Z0-9_]* ", "")

        # Keeping only the word after the #
        X = X.str.replace("#", "")
        X = X.str.replace(r"[-\.\n]", "")
        # Removing HTML garbage
        X = X.str.replace(r"&\w+;", "")
        # Removing links
        X = X.str.replace(r"https?://\S*", "")
        # replace repeated letters with only two occurences
        # heeeelllloooo => heelloo
        X = X.str.replace(r"(.)\1+", r"\1\1")
        # mark emoticons as happy or sad
        #X = X.str.replace(HAPPY_EMO, " happyemoticons ")
        #X = X.str.replace(SAD_EMO, " sademoticons ")
        #X = X.str.lower()
        return X
sentiments = data['Sentiment']
tweets = data['SentimentText']

vectorizer = TfidfVectorizer(tokenizer=lemmatize_tokenize, ngram_range=(1,2))
pipeline = Pipeline([
    ('text_pre_processing', TextPreProc(use_mention=True)),
    ('vectorizer', vectorizer),
])
#  split data into learning set and testing set
learn_data, test_data, sentiments_learning, sentiments_test = train_test_split(tweets, sentiments, test_size=0.3)
learning_data = pipeline.fit_transform(learn_data)

lr = LogisticRegression()
bnb = BernoulliNB()
mnb = MultinomialNB()

models = {
    'logitic regression': lr,
    'bernoulliNB': bnb,
    'multinomialNB': mnb,
}
grid_search_pipeline = Pipeline([
    ('text_pre_processing', TextPreProc()),
    ('vectorizer', TfidfVectorizer()),
    ('model', MultinomialNB()),
])

params = [
    {
        'text_pre_processing__use_mention': [True, False],
        'vectorizer__max_features': [1000, 2000, 5000, 10000, 20000, None],
        'vectorizer__ngram_range': [(1,1), (1,2)],
    },
]
grid_search = GridSearchCV(grid_search_pipeline, params, cv=5, scoring='f1')
grid_search.fit(learn_data, sentiments_learning)
print(grid_search.best_params_)
mnb.fit(learning_data, sentiments_learning)
testing_data = pipeline.transform(test_data)

data0328=pd.read_csv('./20200328 Coronavirus Tweets.CSV', encoding='ISO-8859-1',low_memory=False)
sub_learning= pipeline.transform(data0328['SentimentText'].apply(lambda sub_learning: np.str_(sub_learning05)))

sub = pd.DataFrame(sub_data['ItemID'], columns=("ItemID", "Sentiment"))
sub["Sentiment"] = mnb.predict(sub_learning)
print(sub)

model = MultinomialNB()
model.fit(learning_data, sentiments_learning)
tweet = pd.Series([input(),])
tweet = pipeline.transform(tweet)
proba = model.predict_proba(tweet)[0]
print("The probability that this tweet is sad is:", proba[0])
print("The probability that this tweet is happy is:", proba[1])


NameError: name 'lemmatize_tokenize' is not defined

In [2]:
import pandas as pd
data0305=pd.read_csv('./20200305 Coronavirus Tweets.CSV', encoding='ISO-8859-1',low_memory=False)
data0305

,ItemID,SentimentText,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13
0,1,#CoronaVirusIndonesia \n#CoronaBukaBorokRezim ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,#CoronaVirusIndonesia \n#CoronaBukaBorokRezim ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3,Chinaâs wildlife trade via @SCMPgraphics ht...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4,Proof that the media such as @CNN is unnecessa...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5,Indians are a shitty breed of humans. With the...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
528303,526787,A really great visual explanation of the pande...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
528304,526788,Los casos de COVID-19 se registran mayoritaria...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
528305,526789,Apa itu Virus Coronavirus (Covid-19)?\n\nSumbe...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
528306,526790,Folks upset at the public preparedness measure...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


,ItemID,SentimentText,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13
0,1,#CoronaVirusIndonesia \n#CoronaBukaBorokRezim ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,#CoronaVirusIndonesia \n#CoronaBukaBorokRezim ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3,Chinaâs wildlife trade via @SCMPgraphics ht...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4,Proof that the media such as @CNN is unnecessa...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5,Indians are a shitty breed of humans. With the...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
528303,526787,A really great visual explanation of the pande...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
528304,526788,Los casos de COVID-19 se registran mayoritaria...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
528305,526789,Apa itu Virus Coronavirus (Covid-19)?\n\nSumbe...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
528306,526790,Folks upset at the public preparedness measure...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


,ItemID,SentimentText,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13
0,1,CoronaVirusIndonesia CoronaBukaBorokRezim ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,CoronaVirusIndonesia CoronaBukaBorokRezim ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3,China s wildlife trade via SCMPgraphics ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4,Proof that the media such as CNN is unnecessar...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5,Indians are a shitty breed of humans With the...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
528303,526787,A really great visual explanation of the pande...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
528304,526788,Los casos de COVID se registran mayoritaria...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
528305,526789,Apa itu Virus Coronavirus Covid Sumber ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
528306,526790,Folks upset at the public preparedness measure...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [23]:

import wordcloud
from wordcloud import WordCloud
import matplotlib.pyplot as plt
%matplotlib inline

import re
import nltk
import string
import warnings

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import copy
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os

data = pd.read_csv('./20200305 Coronavirus Tweets.CSV' ,encoding='ISO-8859-1',low_memory=False)
# Removing Twitter Handles (@user)
data['SentimentText'] = data['SentimentText'].str.replace("@", "") 
# Removing links
data['SentimentText'] = data['SentimentText'].str.replace(r"http\S+", "") 
# Removing Punctuations, Numbers, and Special Characters
data['SentimentText'] = data['SentimentText'].str.replace("[^a-zA-Z]", " ") 
# Remove stop words
stopwords=nltk.corpus.stopwords.words('english')
def remove_stopwords(text):
    clean_text=' '.join([word for word in text.split() if word not in stopwords])
    return clean_text
data['SentimentText'] = data['SentimentText'].apply(lambda text : remove_stopwords(text.lower())
data

SyntaxError: invalid syntax (<ipython-input-23-ce69a05e79c4>, line 33)

In [12]:
newdata = copy.deepcopy(data)
newdata.drop(['ItemID'],axis = 1,inplace = True)
all_words = ' '.join([text for text in newdata['SentimentText']])
wordcloud = WordCloud(width=800,height=500,random_state=21,max_font_size=110).generate(all_words)
plt.figure(figsize=(10,7))
plt.imshow(wordcloud, interpolation="bilinear")
plt.axis('off')

plt.show()


TypeError: sequence item 1452: expected str instance, float found